# Submission Notebook

In [1]:
%cd /kaggle
!cp -r /kaggle/input/nfldata-dcnv2/DCNv2 /kaggle/.
%cd /kaggle/DCNv2
! chmod a+x *
! ./make.sh &> /dev/null

/kaggle
/kaggle/DCNv2


In [2]:
# Install motmetrics
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/py-cpuinfo-8.0.0.tar.gz
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/xmltodict-0.12.0-py2.py3-none-any.whl
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/pytest_benchmark-3.4.1-py2.py3-none-any.whl
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/flake8_import_order-0.18.1-py2.py3-none-any.whl
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/motmetrics-1.2.0-py3-none-any.whl

In [3]:
%cd /kaggle
!cp -r /kaggle/input/nfldata-fairmot-package/FairMOT /kaggle/.
%cd /kaggle/FairMOT
! pip install -qq --no-deps --no-index --find-links=/kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install -r requirements.txt #&> /dev/null

/kaggle
/kaggle/FairMOT


In [4]:
! pip uninstall -y datasets

Found existing installation: datasets 1.12.1
Uninstalling datasets-1.12.1:
  Successfully uninstalled datasets-1.12.1


In [5]:
# User
! cp -r /kaggle/input/nfl-helmet-assignment-kaggle/nfl_helmet_assignment_kaggle /kaggle/.

In [6]:
import os, sys
import subprocess
from multiprocessing import Pool
import random
import numpy as np
import pandas as pd
import torch
from IPython.display import display, Video
from tqdm.auto import tqdm

# FairMOT imports
sys.path.append('/kaggle/DCNv2')
sys.path.append('/kaggle/FairMOT/src/lib')
from datasets.dataset.jde import LoadVideo

# nfl_helmet_assignment_kaggle imports
sys.path.append('/kaggle/input/easydict-master/easydict-master')
sys.path.append(
    ('/kaggle/input/yolov5-deepsort-pytorch/Yolov5_DeepSort_Pytorch-master'
     '/Yolov5_DeepSort_Pytorch-master/deep_sort_pytorch')
)
sys.path.append('/kaggle/nfl_helmet_assignment_kaggle/')
from helmet_tracker.utils.score import check_submission
from helmet_tracker.utils.video import ffmpeg_friendly_encode
from helmet_tracker.utils.features import add_track_features
from helmet_tracker.models.helmet_mapping import mapping_df_2d
from helmet_tracker.models.fairmot_postprocess import fmot_postprocess_hmap

In [7]:
# For traindebug videos

# BASE_DIR = '/kaggle/input/nfl-health-and-safety-helmet-assignment'

# dir_video = os.path.join(BASE_DIR, 'train')
# tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
# helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')

# # For developing, restrict to "traindebug" samples

# pth_ss_hmap = '/kaggle/input/nfl-mydata/submission_helmtrack_debug.csv'
# ss_hmap = pd.read_csv(pth_ss_hmap)
# videos = ss_hmap['video_frame'].str.split('_').str[:3].str.join('_').unique()[:2]

# helmets['video'] = helmets['video_frame'].str.split('_').str[:3].str.join('_')
# helmets = helmets[helmets['video'].isin(videos)]

In [8]:
BASE_DIR = '/kaggle/input/nfl-health-and-safety-helmet-assignment'

dir_video = os.path.join(BASE_DIR, 'test')
tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

videos = [pth.split('.')[0] for pth in os.listdir(dir_video)]

helmets['video'] = helmets['video_frame'].str.split('_').str[:3].str.join('_')
helmets = helmets[helmets['video'].isin(videos)]

In [9]:
videos

['57906_000718_Sideline',
 '57906_000718_Endzone',
 '57995_000109_Endzone',
 '58102_002798_Sideline',
 '57995_000109_Sideline',
 '58102_002798_Endzone']

# Helmet mapping 2D

In [10]:
# Add estimated video frame to NGS data

tracking = add_track_features(tracking) 

In [11]:
CONF_THRE = 0.3

In [12]:
df_list = list(helmets.groupby('video_frame'))
df_list = [(video_frame, df, tracking, CONF_THRE) for video_frame, df in df_list]

def _mapping_df(args):
    video_frame, df, tracking, conf_thre = args
    return mapping_df_2d(video_frame, df, tracking, conf_thre=conf_thre)

p = Pool(processes=os.cpu_count())
ss_hmap = []
with tqdm(total=len(df_list)) as pbar:
    for this_df in p.imap(_mapping_df, df_list):
        ss_hmap.append(this_df)
        pbar.update(1)
p.close()

ss_hmap = pd.concat(ss_hmap)

# ss_hmap.to_csv('/kaggle/working/ss_hmap.csv', index=False)

  0%|          | 0/2664 [00:00<?, ?it/s]

# FairMOT

In [13]:
! mkdir -p /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/models-fairmot/dla34-ba72cf86.pth /root/.cache/torch/hub/checkpoints/.

In [14]:
load_model = '/kaggle/input/nfldata-fairmot/exp/mot/nfl_ft_mix_dla34/model_last.pth'
conf_thres = 0.4

dir_demo = '/kaggle/FairMOT/demo'

In [15]:
for video in tqdm(videos, total=len(videos)):
    
    input_video = os.path.join(dir_video, f'{video}.mp4')
    output_root = os.path.join(dir_demo, video)

    ! mkdir -p {output_root}
    %cd /kaggle/FairMOT/src
    ! python demo.py mot --load_model {load_model} --conf_thres {conf_thres} --input-video {input_video} --output-root {output_root}

#     input_path = os.path.join(output_root, 'MOT16-03-results.mp4') 
#     output_path = os.path.join(output_root, 'results.mp4')
#     ffmpeg_friendly_encode(input_path, output_path)
#     os.remove(input_path)
    

  0%|          | 0/6 [00:00<?, ?it/s]

/kaggle/FairMOT/src
Fix size testing.
training chunk_sizes: [6, 6]
The output will be saved to  /kaggle/FairMOT/src/lib/../../exp/mot/default
heads {'hm': 1, 'wh': 4, 'id': 128, 'reg': 2}
2021-11-01 16:26:00 [INFO]: Starting tracking...
Lenth of the video: 440 frames
Creating model...
loaded /kaggle/input/nfldata-fairmot/exp/mot/nfl_ft_mix_dla34/model_last.pth, epoch 16
2021-11-01 16:26:10 [INFO]: Processing frame 0 (100000.00 fps)
2021-11-01 16:26:15 [INFO]: Processing frame 20 (12.65 fps)
2021-11-01 16:26:19 [INFO]: Processing frame 40 (12.72 fps)
2021-11-01 16:26:23 [INFO]: Processing frame 60 (12.50 fps)
2021-11-01 16:26:27 [INFO]: Processing frame 80 (12.51 fps)
2021-11-01 16:26:31 [INFO]: Processing frame 100 (12.40 fps)
2021-11-01 16:26:36 [INFO]: Processing frame 120 (12.40 fps)
2021-11-01 16:26:40 [INFO]: Processing frame 140 (12.37 fps)
2021-11-01 16:26:44 [INFO]: Processing frame 160 (12.27 fps)
2021-11-01 16:26:48 [INFO]: Processing frame 180 (12.20 fps)
2021-11-01 16:26:53

In [16]:
# ! cp -r {dir_demo} /kaggle/working/.

In [17]:
# for video in videos:
#     output_path = os.path.join(dir_demo, video, 'results.mp4')
#     display(
#             Video(data=output_path, embed=True, height=450, width=800)
#         )

# Post-process helmet mapping with FairMOT

In [18]:
video_frame = ss_hmap['video_frame'].str.split('_')
ss_hmap['video'] = video_frame.str[:3].str.join('_')
ss_hmap['frame'] = video_frame.str[3].astype(int)

In [19]:
%%time

outs = []
for video in tqdm(videos, total=len(videos)):
    
    df_hmap = ss_hmap.query('video==@video')

    pth_fmot = os.path.join(dir_demo, video, 'results.txt')
    pth_video = os.path.join(dir_video, video + '.mp4')

    df_tracks = fmot_postprocess_hmap(df_hmap, pth_fmot, pth_video)
    
    outs.append(df_tracks)
    
ss_fmot = pd.concat(outs, axis=0)

  0%|          | 0/6 [00:00<?, ?it/s]

CPU times: user 13.2 s, sys: 78.4 ms, total: 13.3 s
Wall time: 13.3 s


In [20]:
columns = pd.read_csv(
    '/kaggle/input/nfl-health-and-safety-helmet-assignment/sample_submission.csv').columns.values

ss_fmot['label_fairmot'].fillna(ss_fmot['label'], inplace=True)

ss_fmot.drop('label', axis=1, inplace=True)
ss_fmot.rename(columns={'label_fairmot': 'label'}, inplace=True)

ss_fmot = ss_fmot[columns]

is_duped = ss_fmot.duplicated(subset=['video_frame', 'label'], keep='first')
ss_fmot = ss_fmot[~is_duped]


# Check submission and save to disk

In [21]:
check_submission(ss_fmot)

True

In [22]:
ss_fmot.to_csv('/kaggle/working/submission.csv', index=False)